In [2]:
import os
from pathlib import Path
if os.getcwd().endswith("/dev"):
    os.chdir(Path(os.getcwd()) / Path("../backend/src"))


In [3]:
from offspot_metrics_backend.db.models import IndicatorDimension, IndicatorPeriod, IndicatorRecord

from sqlalchemy import desc, func, select, delete

Print query to select top contents

In [4]:
start_ts = 12
stop_ts = 13

subquery = (
            select(
                IndicatorDimension.value0.label("content"),
                func.sum(IndicatorRecord.value).label("count"),
            )
            .join(IndicatorRecord)
            .join(IndicatorPeriod)
            .where(IndicatorPeriod.timestamp >= start_ts)
            .where(IndicatorPeriod.timestamp <= stop_ts)
            .group_by("content")
        ).subquery("content_with_count")

query = select(subquery.c.content).order_by(desc(subquery.c.count)).limit(50)
print(query)


SELECT content_with_count.content 
FROM (SELECT indicator_dimension.value0 AS content, sum(indicator_record.value) AS count 
FROM indicator_dimension JOIN indicator_record ON indicator_dimension.id = indicator_record.dimension_id JOIN indicator_period ON indicator_period.timestamp = indicator_record.period_id 
WHERE indicator_period.timestamp >= :timestamp_1 AND indicator_period.timestamp <= :timestamp_2 GROUP BY content) AS content_with_count ORDER BY content_with_count.count DESC
 LIMIT :param_1


Delete old records

In [5]:
min_ts = 12
query = delete(IndicatorRecord).where(IndicatorRecord.period_id.in_( select(IndicatorPeriod.timestamp).where(IndicatorPeriod.timestamp < min_ts)))
print(query)

DELETE FROM indicator_record WHERE indicator_record.period_id IN (SELECT indicator_period.timestamp 
FROM indicator_period 
WHERE indicator_period.timestamp < :timestamp_1)


Delete dimensions not used anymore

In [6]:
query = delete(IndicatorDimension).where(
                IndicatorDimension.id.in_(select(IndicatorRecord.dimension_id).distinct())
            )
print(query)

DELETE FROM indicator_dimension WHERE indicator_dimension.id IN (SELECT DISTINCT indicator_record.dimension_id 
FROM indicator_record)
